<a href="https://colab.research.google.com/github/Fomalhautwx/Beamformers-Implementation/blob/main/notebooks/BSS/IVA/AuxIVA-IPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ssspy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.io import wavfile

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
n_sources = 2
max_duration = 10

n_fft, hop_length = 4096, 2048

In [15]:
sr, waveform_mix = wavfile.read("/content/drive/MyDrive/Original.wav") # (n_channels, n_sources, n_samples)


In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [10]:
from ssspy.bss.iva import AuxIVA

In [11]:
def contrast_fn(y):
    return 2 * np.linalg.norm(y, axis=1)

def d_contrast_fn(y):
    return 2 * np.ones_like(y)

In [16]:
iva = AuxIVA(
    spatial_algorithm="IPA",
    contrast_fn=contrast_fn,
    d_contrast_fn=d_contrast_fn,
    newton_iter=10,
)
print(iva)

AuxIVA(spatial_algorithm=IPA, scale_restoration=True, record_loss=True, reference_id=0)


In [18]:
print(waveform_mix.shape)
_, _, spectrogram_mix = ss.stft(waveform_mix.T, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

(64000, 2)


In [19]:
spectrogram_est = iva(spectrogram_mix, n_iter=50)

In [20]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(iva.loss)
plt.show()
plt.close()

In [22]:
wavfile.write("/content/drive/MyDrive/Transfer/estimated.wav", sr, waveform_est.T)